In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In this kernel i will be performing sentiment prediction for 'Corona Tweets'  using bi-directional LSTM. The kernel will be having these section and will be explaining it as you scroll

*   Importing the required library's
*   Cleaning the data
  *   Converting the text to lower case
  *   Removing URL's
  *   Removing stop words
  *   Removing punctuation's

*   Preparing the text and building the model
  *   Tokenizing the text
  *   Converting the texts to sequences
  *   Padding the sequence
  *   Building the model & model summary
  *   Training the model


*   Predicting on test data and calculating accuracy score and plotting confusion matrix







<h3>Importing required library's</h3>

In [ ]:
import tensorflow.keras.layers as L
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
import nltk
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding="latin-1")

In [ ]:
test_data = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv")


In [ ]:
df.head()


In [ ]:
df['Sentiment'].value_counts()


Since we have unique values like 'Positive' , 'Negative' , 'Neutral' , 'Extremely Positive' , ' Extremely Negative' we will just convert them to 'Positive' , 'Negative, 'Neutral' with it's corresponding integer value

In [ ]:
def encode_sentiment(sentiment):
  if sentiment=="Neutral":
    return 0
  elif sentiment=="Positive":
    return 2
  elif sentiment=="Negative":
    return 1
  elif sentiment=="Extremely Positive":
    return 2
  elif sentiment=="Extremely Negative":
    return 1

Creating a new column called 'new_sentiment' and applying the 'encode_sentiment' function to the 'sentiment' column which results in encoding the values to an integer

In [ ]:
df['new_sentiment'] = df['Sentiment'].apply(encode_sentiment)


In [ ]:
df['new_sentiment'].value_counts()


In [ ]:
train_df = df[['OriginalTweet','new_sentiment']]


In [ ]:
train_df.head()


In [ ]:
train_df = train_df.sample(frac=1)


In [ ]:
sentiment = train_df['new_sentiment'].values


<H3>Preprocessing Text</h3>

For Preprocessing we will , 
*   Convert the text to string
*   Lower the string
*   Create a Regular Expression to remove any URL's
*   Removing punctuation
*   Removing any special characters
*   Remove stopwords

In [ ]:
def process_text(text):
  text = str(text) #Convert string to str
  #Lowers the string
  text = text.lower()
  #Removes the full url
  url_remove = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
  text = re.sub(url_remove,' ',text)
  #Removes the punctuation
  text = ''.join([string for string in text if string not in punctuation and not string.isdigit()])
  #Removes any more special characters
  special_character = re.compile(r'[^a-zA-Z]')
  text = re.sub(special_character,' ', text)
  text = text.strip() #Strip white spaces
  text = text.split(' ')
  text = ' '.join([string for string in text if string not in stopwords.words('english')])#Removing all stop words
  return text

In [ ]:
train_df['processed_tweet'] = train_df['OriginalTweet'].apply(process_text)

In [ ]:
train_df.head()

<h3>Tokenizing the text</h3>

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(train_df['processed_tweet'].values)

Converting the text into integer sequence

In [ ]:
train =  tokenizer.texts_to_sequences(train_df['processed_tweet'].values)

Padding post the text with '0' so they all be of same length so we can fit them into the model

In [ ]:
train = pad_sequences(train,padding='post')

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(train,sentiment,test_size=0.2,random_state=42)

<h3>Building the Model</h3>

In [ ]:
embedding_vectors = 30
VOCAB_SIZE = len(tokenizer.word_index)+1

In [ ]:
model = tf.keras.Sequential([
    L.Embedding(VOCAB_SIZE,embedding_vectors, input_length=x_train.shape[1]),
    L.Bidirectional(L.LSTM(256,return_sequences=True)),
    L.GlobalMaxPool1D(),
    L.Dropout(0.4),
    L.Dense(256, activation="relu"),
    L.Dropout(0.4),
    L.Dense(3)
])

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',metrics=['accuracy']
             )

In [ ]:
model.summary()

<h3>Training the data</h3>

In [ ]:
model.fit(x_train,y_train,epochs=2,
          validation_data=(x_test,y_test))

<h3>Making predictions on test data</h3>

In [ ]:
test = test_data[['OriginalTweet','Sentiment']]

In [ ]:
test['processed_tweet'] = test_data['OriginalTweet'].apply(process_text)

In [ ]:
test['new_sentiment'] = test_data['Sentiment'].apply(encode_sentiment)

In [ ]:
test.drop(['OriginalTweet','Sentiment'],inplace=True,axis=1)

In [ ]:
test.head()

In [ ]:
test_tweet = test['processed_tweet'].values
test_sentiment = test['new_sentiment'].values

In [ ]:
convert_seq = tokenizer.texts_to_sequences(test_tweet)

In [ ]:
convert_seq = pad_sequences(convert_seq,padding='post')

<h3>Making predictions on test data</h3>

In [ ]:
predict = model.predict_classes(convert_seq)

In [ ]:
print(f"The accuracy is : { accuracy_score(test_sentiment,predict)*100}%")

<h3>Confusion Matrix</h3>

In [ ]:
sns.heatmap(confusion_matrix(test_sentiment,predict),annot=True, fmt="d")

Where,
*   0 - Neutral
*   1 - Negative
*   2 - Positive


If you liked this kernel , Please give it an upvote! :) 